# Make postISRCCD from raw for Spectractor


- work with Weakly_2022_39
- use jupyter kernel LSST : **lsst_distrib_2022_39**



- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/12/08
- last update : 2022/12/09



In [1]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2022_39/atmospec 	setup
drp_pipe              LOCAL:/home/d/dagoret/repos/repos_w_2022_39/drp_pipe 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-4.1.0/eups 	setup
summit_utils          LOCAL:/home/d/dagoret/repos/repos_w_2022_39/summit_utils 	setup


In [2]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

w_2022_39
   g0b29ad24fb+a10408d0bf 	current w_2022_39 setup


In [3]:
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates

import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch


from astropy.io import fits




In [4]:
# Assembly task
# https://github.com/lsst/ip_isr/blob/main/python/lsst/ip/isr/isrTask.py

from lsst.ip.isr.assembleCcdTask import (AssembleCcdConfig, AssembleCcdTask)
from lsst.ip.isr.isrTask import (IsrTask, IsrTaskConfig)

#https://github.com/lsst/ip_isr/blob/main/python/lsst/ip/isr/overscan.py
from lsst.ip.isr import  OverscanCorrectionTaskConfig, OverscanCorrectionTask

In [5]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [6]:
import lsst.daf.butler as dafButler

# Configuration

In [7]:
FLAG_PLOT=False

In [8]:
DATE = 20221207
FILTER="empty-holo4_003"
#filename_visits = f"visitdispersers_{DATE}_filt_{FILTER}.list"
filename_visits = f"all_visitdispersers/{DATE}/visitdispersers_{DATE}_filt_{FILTER}.list"

In [9]:
top_path_out="my_postisrccd_img"
path_out=f"{top_path_out}/{DATE}"

In [10]:
if not os.path.exists(path_out):
    os.makedirs(path_out)

# read list of exposures

- generated by ListOfExposures-hologram.ipynb

In [11]:
df = pd.read_csv(filename_visits, header=None,skiprows = 1, sep=' ',names=["date","seq"])
df

,date,seq
0,20221207,357
1,20221207,361
2,20221207,362
3,20221207,366
4,20221207,367
5,20221207,371
6,20221207,372
7,20221207,376
8,20221207,377
9,20221207,381


In [12]:
for index,row in df.iterrows():
    exposure_selected =row["date"]*100000+row["seq"]
    print(exposure_selected)

2022120700357
2022120700361
2022120700362
2022120700366
2022120700367
2022120700371
2022120700372
2022120700376
2022120700377
2022120700381
2022120700382
2022120700386
2022120700387
2022120700391
2022120700392
2022120700396
2022120700397
2022120700401
2022120700402
2022120700406
2022120700407
2022120700411
2022120700412
2022120700416
2022120700417
2022120700421
2022120700422
2022120700426
2022120700427
2022120700431
2022120700432
2022120700436
2022120700437
2022120700442
2022120700443
2022120700447
2022120700448
2022120700453
2022120700454
2022120700458
2022120700459
2022120700463
2022120700464


## Select flags options

In [13]:
FLAG_ROTATE_IMG = True
FLAG_TRANSFORM = True

## Transformations
astropy scale transformations

In [14]:
transform = AsinhStretch() + PercentileInterval(99.)
#transform = PercentileInterval(98.)

## Butler

In [15]:
#repo = '/sdf/group/rubin/repo/main'
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [16]:
collection='LATISS/raw/all'

In [17]:
# configuration
isr_config =  IsrTaskConfig()

In [18]:
isr_config.doDark = False
isr_config.doFlat =  False
isr_config.doFringe = False
isr_config.doDefect = True
isr_config.doLinearize = False
isr_config.doCrosstalk =  False
isr_config.doSaturationInterpolation = False
isr_config.overscan.fitType: 'MEDIAN_PER_ROW'
isr_config.doBias: True


In [19]:
isr_task = IsrTask(config=isr_config)

In [20]:
calibType = 'bias'
physical_filter = 'empty~empty'
cameraName = 'LATISS'
# Collection name containing the verification outputs.
calibCollections = ['LATISS/calib','LATISS/raw/all',
                   'LATISS/calib/DM-28636',
'LATISS/calib/DM-28636/curated/19700101T000000Z',
'LATISS/calib/DM-28636/curated/20180101T000000Z',
'LATISS/calib/DM-28636/unbounded',
'LATISS/calib/DM-33875',
'LATISS/calib/DM-36484/bias.20221005a',
'LATISS/calib/DM-36484/biasGen.20221005a',
'LATISS/calib/DM-36484/biasGen.20221005a/20221006T000101Z',
'LATISS/calib/DM-36484/dark.20221006a',
'LATISS/calib/DM-36484/darkGen.20221005a',
'LATISS/calib/DM-36484/darkGen.20221005a/20221006T222501Z',
'LATISS/calib/DM-36484/darkGen.20221006a',
'LATISS/calib/DM-36484/darkGen.20221006a/20221006T222921Z',
'LATISS/calib/DM-36484/flat-SDSSg.20221006a',
'LATISS/calib/DM-36484/flat-SDSSi.20221006a',
'LATISS/calib/DM-36484/flat-SDSSr.20221006a',
'LATISS/calib/DM-36484/flatGen-SDSSg.20221006a',
'LATISS/calib/DM-36484/flatGen-SDSSg.20221006a/20221007T002703Z',
'LATISS/calib/DM-36484/flatGen-SDSSi.20221006a',
'LATISS/calib/DM-36484/flatGen-SDSSi.20221006a/20221007T003732Z',
'LATISS/calib/DM-36484/flatGen-SDSSiD.20221006a',
'LATISS/calib/DM-36484/flatGen-SDSSiD.20221006a/20221007T004708Z',
'LATISS/calib/DM-36484/flatGen-SDSSr.20221006a',
'LATISS/calib/DM-36484/flatGen-SDSSr.20221006a/20221006T233657Z',
'LATISS/calib/DM-36484/verifyBias.20221005a',
'LATISS/calib/DM-36484/verifyBias.20221005a/20221006T000747Z',
'LATISS/calib/DM-36484/verifyBias.20221005a/20221006T213237Z',
'LATISS/calib/DM-36484/verifyBias.20221005b',
'LATISS/calib/DM-36484/verifyBias.20221005b/20221019T205236Z',
'LATISS/calib/DM-36484/verifyDark.20221006a',
'LATISS/calib/DM-36484/verifyDark.20221006a/20221006T224403Z',
'LATISS/calib/DM-36484/verifyFlat-SDSSg.20221006a',
'LATISS/calib/DM-36484/verifyFlat-SDSSg.20221006a/20221007T003418Z',
'LATISS/calib/DM-36484/verifyFlat-SDSSi.20221006a',
'LATISS/calib/DM-36484/verifyFlat-SDSSi.20221006a/20221007T004423Z',
'LATISS/calib/DM-36484/verifyFlat-SDSSr.20221006a',
'LATISS/calib/DM-36484/verifyFlat-SDSSr.20221006a/20221006T234341Z',
'LATISS/calib/DM-36719',
'LATISS/calib/DM-36719/bias.20221107',
'LATISS/calib/DM-36719/biasGen.20221107a',
'LATISS/calib/DM-36719/biasGen.20221107a/20221107T205127Z',
'LATISS/calib/DM-36719/biasGen.20221107b',
'LATISS/calib/DM-36719/biasGen.20221107b/20221107T213306Z',
'LATISS/calib/DM-36719/dark.20221107',
'LATISS/calib/DM-36719/darkGen.20221107a',
'LATISS/calib/DM-36719/darkGen.20221107a/20221107T223409Z',
'LATISS/calib/DM-36719/flat-SDSSg.20221107',
'LATISS/calib/DM-36719/flat-SDSSi.20221107',
'LATISS/calib/DM-36719/flat-SDSSr.20221107',
'LATISS/calib/DM-36719/flatGen-SDSSg.20221107a',
'LATISS/calib/DM-36719/flatGen-SDSSg.20221107a/20221108T002737Z',
'LATISS/calib/DM-36719/flatGen-SDSSi.20221107a',
'LATISS/calib/DM-36719/flatGen-SDSSi.20221107a/20221108T005202Z',
'LATISS/calib/DM-36719/flatGen-SDSSr.20221107a',
'LATISS/calib/DM-36719/flatGen-SDSSr.20221107a/20221107T235401Z',
'LATISS/calib/DM-36719/ptcGen-SDSSr.20221107a',
'LATISS/calib/DM-36719/ptcGen-SDSSr.20221107a/20221108T180421Z',
'LATISS/calib/DM-36719/verifyBias.20221107b',
'LATISS/calib/DM-36719/verifyBias.20221107b/20221107T220410Z',
'LATISS/calib/DM-36719/verifyDark.20221107a',
'LATISS/calib/DM-36719/verifyDark.20221107a/20221107T232823Z',
'LATISS/calib/DM-36719/verifyFlat-SDSSg.20221107a',
'LATISS/calib/DM-36719/verifyFlat-SDSSg.20221107a/20221108T004225Z',
'LATISS/calib/DM-36719/verifyFlat-SDSSi.20221107a',
'LATISS/calib/DM-36719/verifyFlat-SDSSi.20221107a/20221108T012110Z',
'LATISS/calib/DM-36719/verifyFlat-SDSSi.20221107a/20221108T014950Z',
'LATISS/calib/DM-36719/verifyFlat-SDSSr.20221107a',
'LATISS/calib/DM-36719/verifyFlat-SDSSr.20221107a/20221108T000940Z',
'LATISS/calib/unbounded','LATISS/defaults','LATISS/raw/all']

In [21]:
butler = dafButler.Butler(repo, collections=calibCollections)
camera = butler.get('camera', instrument=cameraName)
bias = butler.get('bias',instrument=cameraName,detector=0)
defects = butler.get('defects',instrument=cameraName,detector=0)


In [22]:
if FLAG_PLOT:
    for index,row in df.iterrows():
        exposure_selected =row["date"]*100000+row["seq"]



        raw_img= butler.get('raw', dataId={'exposure': exposure_selected, 'instrument': 'LATISS', 'detector': 0}, collections = collection)
        meta = raw_img.getMetadata()
        md = meta.toDict()
    
        the_object = md['OBJECT']
        the_am= md['AMSTART']
        the_filter=md['FILTER']
    
        #fast ISR 
        isr_img = isr_task.run(raw_img,bias=bias,defects=defects)
    
        rotated_array = isr_img.exposure.image.array[::-1,::-1] #rotate the array 180 degrees
        #np.flip(np.flip(a, 1), 0)

    
        fig = plt.figure(figsize=(12,10))
        afw_display = afwDisplay.Display(frame=fig)
        afw_display.scale('linear', 'zscale',None)
        the_title = f"{index} :: isr exposure : {exposure_selected}, target={the_object}, airmass={the_am:.2f}, filter={the_filter}"
        afw_display.mtv(isr_img.exposure.image,title=the_title)

In [23]:
# Save in files

In [24]:
for index,row in df.iterrows():
    
    exposure_selected =row["date"]*100000+row["seq"]



    raw_img= butler.get('raw', dataId={'exposure': exposure_selected, 'instrument': 'LATISS', 'detector': 0}, collections = collection)
    isr_img = isr_task.run(raw_img,bias=bias,defects=defects)
    
    arr=isr_img.exposure.image.array
    # 180 degree rotation
    rotated_array = arr[::-1,::-1] #rotate the array 180 degrees
    
    
    meta = raw_img.getMetadata()
    md = meta.toDict()

    the_object = md['OBJECT']
    the_am= md['AMSTART']
    the_filter=md['FILTER']

    filename_out = f"exposure_{exposure_selected}_pseudo-postisrccd.fits"
    fullfilename_out=os.path.join(path_out,filename_out)
    
    print(filename_out)
    
    hdr = fits.Header()
    
    for key,value in md.items():
        hdr[str(key)]=value
        
    # need this    
    hdr["AMEND"] = hdr["AMSTART"]
    
    # be aware weather data may be missing
    
        
    hdu = fits.PrimaryHDU(data=rotated_array,header=hdr)  # with headers
    #hdu = fits.PrimaryHDU(data=all_my_raw_array[idx])
    
    hdul = fits.HDUList([hdu])
    
    hdul.writeto(fullfilename_out,overwrite=True)
    





/tmp/tmptz8m9epl.fits: outAmp.getRawBBox() != data.getBBox(); patching. ((minimum=(0, 0), maximum=(543, 2047)) v. (minimum=(0, 0), maximum=(575, 2047)))
No rough magnitude zero point defined for filter empty~holo4_003.


exposure_2022120700357_pseudo-postisrccd.fits


/tmp/tmpzgoyl_e7.fits: outAmp.getRawBBox() != data.getBBox(); patching. ((minimum=(0, 0), maximum=(543, 2047)) v. (minimum=(0, 0), maximum=(575, 2047)))
No rough magnitude zero point defined for filter empty~holo4_003.


exposure_2022120700361_pseudo-postisrccd.fits


/tmp/tmpi7ihd91k.fits: outAmp.getRawBBox() != data.getBBox(); patching. ((minimum=(0, 0), maximum=(543, 2047)) v. (minimum=(0, 0), maximum=(575, 2047)))
No rough magnitude zero point defined for filter empty~holo4_003.


...

exposure_2022120700459_pseudo-postisrccd.fits


/tmp/tmpoaj40sws.fits: outAmp.getRawBBox() != data.getBBox(); patching. ((minimum=(0, 0), maximum=(543, 2047)) v. (minimum=(0, 0), maximum=(575, 2047)))
No rough magnitude zero point defined for filter empty~holo4_003.


exposure_2022120700463_pseudo-postisrccd.fits


/tmp/tmprnfyvf9o.fits: outAmp.getRawBBox() != data.getBBox(); patching. ((minimum=(0, 0), maximum=(543, 2047)) v. (minimum=(0, 0), maximum=(575, 2047)))
No rough magnitude zero point defined for filter empty~holo4_003.


exposure_2022120700464_pseudo-postisrccd.fits


In [25]:
rotated_array.shape

(4000, 4072)